In [1]:
import hana_ml
print(hana_ml.__version__)

2.6.20110600


In [2]:
hana_cloud_endpoint="8e1a286a-21d7-404d-8d7a-8c77d2a77050.hana.trial-eu10.hanacloud.ondemand.com:443"

In [3]:
hana_cloud_host, hana_cloud_port=hana_cloud_endpoint.split(":")

cchc=hana_ml.dataframe.ConnectionContext(port=hana_cloud_port,
                                         address=hana_cloud_host,
                                         user='DBAdmin',
                                         encrypt=True
                                        )

HANA DB User : DBAdmin Password :  ··········


In [4]:
print(cchc.hana_version())
print(cchc.get_current_schema())

4.00.000.00.1602165697 (fa/CE2020.36)
DBADMIN


In [5]:
newuser="HANAML"
newpassword="Super$ecr3t!"

cursor = cchc.connection.cursor()

cursor.execute("DROP USER {} CASCADE".format(newuser))
cursor.execute('CREATE USER {name} PASSWORD "{pwd}" NO FORCE_FIRST_PASSWORD_CHANGE SET USERGROUP DEFAULT'.format(name=newuser, pwd=newpassword))
cursor.execute("ALTER USER {} DISABLE PASSWORD LIFETIME".format(newuser))

cursor.close()

In [6]:
cchc.connection.__class__

hdbcli.dbapi.Connection

In [7]:
print(cchc
      .sql("""SELECT USER_NAME, CREATOR, PASSWORD_CHANGE_NEEDED, IS_PASSWORD_LIFETIME_CHECK_ENABLED 
              FROM USERS 
              WHERE USER_NAME='{}'
              """.format(newuser))
      .collect())

  USER_NAME  CREATOR PASSWORD_CHANGE_NEEDED IS_PASSWORD_LIFETIME_CHECK_ENABLED
0    HANAML  DBADMIN                  FALSE                              FALSE


In [8]:
print(cchc.table("USERS", schema="SYS")
      .select('USER_NAME', 'CREATOR', 'PASSWORD_CHANGE_NEEDED', 'IS_PASSWORD_LIFETIME_CHECK_ENABLED')
      .filter("USER_NAME='{}'".format(newuser))
      .collect())

  USER_NAME  CREATOR PASSWORD_CHANGE_NEEDED IS_PASSWORD_LIFETIME_CHECK_ENABLED
0    HANAML  DBADMIN                  FALSE                              FALSE


In [9]:
print(cchc.sql(cchc
               .table("USERS", schema="SYS")
               .select('USER_NAME', 'CREATOR', 'PASSWORD_CHANGE_NEEDED', 'IS_PASSWORD_LIFETIME_CHECK_ENABLED')
               .filter("USER_NAME='{}'".format(newuser))
               .select_statement)
      .collect())

  USER_NAME  CREATOR PASSWORD_CHANGE_NEEDED IS_PASSWORD_LIFETIME_CHECK_ENABLED
0    HANAML  DBADMIN                  FALSE                              FALSE
